In [1]:
import pandas as pd
import sqlite3

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)


In [2]:
orders = pd.read_csv("orders.csv")

print("Orders Data:")
display(orders.head())
print("Shape:", orders.shape)


Orders Data:


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


Shape: (10000, 6)


In [3]:
users = pd.read_json("users.json")

print("Users Data:")
display(users.head())
print("Shape:", users.shape)


Users Data:


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


Shape: (3000, 4)


In [4]:
# Create in-memory SQLite DB
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Load SQL file
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)

# Load into DataFrame
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

print("Restaurants Data:")
display(restaurants.head())
print("Shape:", restaurants.shape)


Restaurants Data:


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


Shape: (500, 4)


In [5]:
# Merge Orders + Users
merged_1 = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

# Merge with Restaurants
final_data = pd.merge(
    merged_1,
    restaurants,
    on="restaurant_id",
    how="left"
)

print("Final Merged Dataset:")
display(final_data.head())
print("Shape:", final_data.shape)


Final Merged Dataset:


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


Shape: (10000, 12)


In [6]:
final_data["order_date"] = pd.to_datetime(
    final_data["order_date"],
    dayfirst=True
)

print("Date Format Fixed")


Date Format Fixed


In [7]:
final_data.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

print("Final dataset saved successfully!")


Final dataset saved successfully!


In [8]:
final_data.info()
final_data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   order_id           10000 non-null  int64         
 1   user_id            10000 non-null  int64         
 2   restaurant_id      10000 non-null  int64         
 3   order_date         10000 non-null  datetime64[ns]
 4   total_amount       10000 non-null  float64       
 5   restaurant_name_x  10000 non-null  object        
 6   name               10000 non-null  object        
 7   city               10000 non-null  object        
 8   membership         10000 non-null  object        
 9   restaurant_name_y  10000 non-null  object        
 10  cuisine            10000 non-null  object        
 11  rating             10000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 937.6+ KB


,order_id,user_id,restaurant_id,order_date,total_amount,rating
count,10000.00000,10000.000000,10000.000000,10000,10000.000000,10000.000000
mean,5000.50000,1504.117700,251.016700,2023-07-02 11:45:53.280000,801.162412,4.045430
min,1.00000,1.000000,1.000000,2023-01-01 00:00:00,100.200000,3.000000
25%,2500.75000,761.000000,127.000000,2023-04-01 00:00:00,446.310000,3.500000
50%,5000.50000,1508.000000,251.000000,2023-07-03 00:00:00,806.295000,4.100000
75%,7500.25000,2250.250000,376.000000,2023-10-02 00:00:00,1149.227500,4.600000
max,10000.00000,3000.000000,500.000000,2024-01-01 00:00:00,1499.830000,5.000000
std,2886.89568,861.727776,144.622558,NaN,405.458753,0.606531


In [9]:
final_data["membership"].value_counts()


membership
Regular    5013
Gold       4987
Name: count, dtype: int64

In [10]:
city_revenue = final_data.groupby("city")["total_amount"].sum().sort_values(ascending=False)

city_revenue


city
Bangalore    2206946.58
Chennai      1990513.03
Pune         1924797.93
Hyderabad    1889366.58
Name: total_amount, dtype: float64

In [11]:
cuisine_orders = final_data.groupby("cuisine")["order_id"].count().sort_values(ascending=False)

cuisine_orders


cuisine
Mexican    2581
Italian    2532
Indian     2469
Chinese    2418
Name: order_id, dtype: int64

In [12]:
final_data.groupby("membership")["total_amount"].mean()


membership
Gold       797.145556
Regular    805.158434
Name: total_amount, dtype: float64

In [13]:
monthly_revenue = final_data.groupby(
    final_data["order_date"].dt.month
)["total_amount"].sum()

monthly_revenue


order_date
1     663020.26
2     630867.40
3     716738.98
4     666097.18
5     668428.61
6     610822.93
7     688559.45
8     694987.58
9     653838.07
10    699187.73
11    637772.43
12    681303.50
Name: total_amount, dtype: float64

In [14]:
top_users = final_data.groupby("user_id")["total_amount"].sum().sort_values(ascending=False).head(10)

top_users


user_id
1515    11556.49
650     10747.44
496      9634.30
2586     9486.61
2615     9237.32
636      9087.09
2973     9063.07
874      8916.55
1337     8907.50
2694     8875.71
Name: total_amount, dtype: float64